In [1]:
#!pip install dash==2.6.0
#!pip install dash_bootstrap_components
#!pip install jupyter_dash


In [2]:
import datetime
import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import re
import tensorflow as tf
import json
import plotly.graph_objects as go
import joblib
import math
from keras.models import load_model

import plotly.express as px
from dash import Dash, dcc, html, Input, Output, State
import dash_bootstrap_components as dbc
from jupyter_dash import JupyterDash

import sys
sys.path.append('./rtaUtils')
from rtaUtils import data_loading, common, sort_vectors, paths, experiment, data_preparation


config = {
  'toImageButtonOptions': {
    'format': 'png', # one of png, svg, jpeg, webp
    'filename': 'newplot',
    'height': 1000,
    'width': 1000,
    'scale':1 # Multiply title/legend/axis/canvas sizes by this factor
  }
}

airport_list = ('LEBL','LPPT','LFPO','LFPG','EGLL','LPPR','EDDF','EBBR','EHAM','LEBB','LIRF',
                'EDDM','EGKK','LEVC','LSGG','LECO','LIMC','EIDW','LFML','LEZL','LOWW','LIPZ',
                'LIPE','LFLL','EDDL','EDDB','LTFM','LFMN','LFRS','LROP','EDDP','LGAV','EDDH',
                'LHBP','EKCH','EGCC','ELLX','LKPR','LIRN','LBSF','EPWA')

"""
paths.airports_file_path
aeropuertos = pd.read_csv("data/airports.csv")
listaAeropuertos = aeropuertos["id"].unique()
"""
"""
numeric_feat   = ['latitude', 'longitude', 'altitude'] 
categoric_feat = [] #'operator'      
objective      = ['latitude', 'longitude', 'altitude']
"""

numeric_feat   = ["latitude", "longitude", "altitude",'track','sector','hav_distance','speed','vspeed']
categoric_feat = ["aerodromeOfDeparture", "aerodromeOfDestination"]  
objective      = ['latitude', 'longitude', 'altitude']

feat_dict = dict(
    numeric=numeric_feat,
    categoric=categoric_feat,
    objective=objective
)

# Cosas TSMixer

In [3]:
### Data configuration
sampling = 15
months = "202201"
# months = '*' # 20220[123456789]

# Para que el label encoder tengo todos los aeropuertos
aeropuertos = pd.read_csv("data/airports.csv")
listaAeropuertos = aeropuertos["id"].unique()

### Model configuration

#lookback     = 55
lookforward  = 10
shift        = 0
#batch_size   = 32

checkpoints_path = 'models/'

### Global variables for runtime
month_data = None           # Dataframe donde precargar los datos del mes
tray_data = None            # Dataframe para precargar los datos de la trayectoria elegida
prediction = None           # Array para precargar las predicciones para la trayectoria completa
windows = None              # Lista para almacenar las ventanas generadas a partir de la trayectoria
prediction_windows = None   # Lista para almacenar las predicciones generadas a partir de las ventanas
true_windows = None         # Lista para almacenar los valores reales correspondientes a las ventanas

current_month = '202201'
models = [x.name for x in Path('./models').glob('*/')]

modelo = None
nombre_modelo = None # Para poder saber de qué modelo cargar la versión
configModelo = None
encoder = None
scalers = None

In [4]:
def get_windows(dataframe: pd.DataFrame, lookback: int):
    """Generate sequences of size lookback of adjacent vectors

    Windows with vectors from two or more different segments are discarded.

    Args:
        dataframe: Trajectories data
        lookback: Length of the sliding window
        encoders: Scikit-learn Encoder object for the passed categoric features
        scaler: Scikit-learn Scaler object for the passed feature configuration
        features: Dictionary with list of strings identifying features of
            each type:
            { numeric:[feat1, ...], categoric:[...], objective:[...] }
    """
    numeric_feat = feat_dict.get('numeric')
    categoric_feat = feat_dict.get('categoric')
    objective = feat_dict.get('objective')
    
    dataframe['segment'] = data_preparation.identify_gaps(dataframe)

    indices = dataframe.reset_index(drop=True).reset_index().groupby(['fpId','segment'])\
                       .aggregate({'index':['min','max']}).reset_index()

    dfColumnas = dataframe[numeric_feat+categoric_feat+objective]

    dataset = [tf.data.Dataset.from_tensor_slices(dfColumnas.iloc[start:end+1,:])
               for f, (fpId, segment, start, end) in list(indices.iterrows())]
    dataset = tf.data.Dataset.from_tensor_slices(dataset)
    dataset = dataset.flat_map(lambda x: x.window(lookback, shift=1, stride=1, drop_remainder=True))
    dataset = dataset.flat_map(lambda window: window.batch(lookback))

    return dataset

def format_data(dataset: tf.data.Dataset, lookback: int) -> tf.data.Dataset:
    """Formatting data for use as input to the model

    Uses window data to construct valed examples for the recurrent model.
    """

    objective_feat = feat_dict.get('objective')
    dataset = dataset.map(lambda x: (x[:lookback, :-len(objective_feat)],
                                  tf.reshape(x[-lookforward:, -len(objective_feat):],
                                              (lookforward, len(objective_feat)))))

    return dataset

def crear_dataset(datos: pd.DataFrame, lookback: int, batch_size: int, randomize: bool = False) -> tf.data.Dataset:
    if randomize:
        aps = sorted(datos.aerodromeOfDeparture.unique())
        counts = datos.aerodromeOfDeparture.value_counts()
        probs = [counts[ap]/len(datos) for ap in aps]
    
        datasets = [get_windows(datos[datos.aerodromeOfDeparture == ap].copy(),
                                lookback + lookforward + shift).shuffle(1000)
                    for ap in aps]
        print(datasets)
        dataset = tf.data.Dataset.sample_from_datasets(datasets, weights=probs)
        
    else:
        dataset = get_windows(datos, lookback + lookforward + shift)

    formated_data = format_data(dataset, lookback)
    batched_data = formated_data.batch(batch_size)

    return batched_data

## Inicialización

In [5]:
# Estilos de los ejemplos de Dash
external_stylesheets = [dbc.themes.BOOTSTRAP,'https://codepen.io/chriddyp/pen/bWLwgP.css']

#app = JupyterDash(__name__, external_stylesheets=external_stylesheets)
app = Dash(__name__, external_stylesheets=external_stylesheets)

## Estructura

In [6]:
############### Model ###############

modelSelector = dcc.Dropdown(
    id = 'modelSelector',
    options={x:x.upper().replace('_', ' ') for x in models},
    value=models[0],
)

modelVersionSelector = dcc.Dropdown(
    id = 'modelVersionSelector',
)

############### Trajectories ###############

dateSelector = dcc.DatePickerSingle(
    id = 'dateSelector',
    placeholder = 'Select one...',
    date='2022-01-01',
#     initial_visible_month=data.flightDate.min(),
    min_date_allowed = '2022-01-01',
    max_date_allowed = '2022-09-30',
    display_format='DD/MM/YYYY',
#     disabled_days=dates
)

airportSelector = dcc.Dropdown(
    id = 'airportSelector',
    options = [dict(label=x, value=x) for x in sorted(airport_list)],
    placeholder='Select one...',
    multi=True,
)

trajectorySelector = dcc.Dropdown(
    id = 'trajectorySelector',
    placeholder='Select one...',
    multi=False,
)

windowSelector = dcc.Input(
    id = 'windowSelector',
    type = 'number',
    min=0, max=100,
    value=0,
    #tooltip={'placement': 'top', 'always_visible': False},
    step=1
)

app.layout = html.Div(style=dict(display='flex'), children=[
    html.Div(style=dict(width='20%', padding=5,), children=[
        html.Fieldset(style=dict(width='99%', borderWidth=1, borderStyle='solid', 
                                 borderRadius=5, padding=5, margin=2), children=[
            html.Legend('Data', style=dict(fontWeight='bold')),
            dbc.Row(style=dict(paddingTop=5, paddingBottom=5), children=[
                dbc.Col(dbc.Label('Months'), width=4),
                dbc.Col(months, width=7),
                dbc.Col(dbc.Label('Sampling'), width=4),
                dbc.Col(sampling, width=7),
            ]),
        ]),
        
        html.Fieldset(style=dict(width='99%', borderWidth=1, borderStyle='solid', 
                                 borderRadius=5, padding=5, margin=2), children=[
            html.Legend('Model', style=dict(fontWeight='bold')),
            dbc.Row(style=dict(paddingTop=5, paddingBottom=5), children=[
                dbc.Col([dbc.Label('Model'), modelSelector], width=12),
            ]),
            dbc.Row(style=dict(paddingTop=5, paddingBottom=5), children=[
                dbc.Col([dbc.Label('Version'), modelVersionSelector], width=12),
            ]),
            html.Div(id='placeholder', style={'display':'none'})
        ]),
        
        html.Fieldset(style=dict(width='99%', borderWidth=1, borderStyle='solid', 
                                 borderRadius=5, padding=5, margin=2), children=[
            html.Legend('Trajectory', style=dict(fontWeight='bold')),
            dbc.Row(style=dict(paddingTop=5, paddingBottom=5), children=[
                dbc.Col([dbc.Label('Date'), dateSelector], width=6),
                dbc.Col([dbc.Label('Airport'), airportSelector], width=6),
            ]),
            dbc.Row(style=dict(paddingTop=5, paddingBottom=5), children=[
                dbc.Col([dbc.Label('Trajectory'), trajectorySelector], width=12),
            ]),
        ]),
        
        html.Fieldset(style=dict(width='99%', borderWidth=1, borderStyle='solid', 
                                 borderRadius=5, padding=5, margin=2), children=[
            html.Legend('Experiment', style=dict(fontWeight='bold')),
            dcc.RadioItems(
                id='experimentTypeSelector',
                style=dict(padding=5),
                options=dict(full=' Full trajectory', window=' Individual window'),
                value='full',
            ),
            html.Div(id='sliderDiv', children=[windowSelector]),
            html.Button('Run experiment', id='runButton', 
                        style=dict(margin='auto', marginTop=20, display='flex', alignItems='center')),
        ]),
        
        html.Fieldset(style=dict(width='99%', borderWidth=1, borderStyle='solid', 
                                 borderRadius=5, padding=5, margin=2), children=[
            html.Legend('Map type', style=dict(fontWeight='bold')),
            dcc.RadioItems(
                id='mapTypeSelector',
                style=dict(padding=5),
                options=['open-street-map',
                         'carto-positron', 
                         'carto-darkmatter', 
                         'stamen-terrain', 
                         'stamen-toner'],
                value='open-street-map',
            ),
        ]),
    ]),
    
    html.Div(id='mapDiv', style=dict(width='75%', margin=7, borderWidth=1, borderStyle='solid', borderRadius=5), children=[
        dcc.Graph(id='mapGraph', style=dict(width='100%',height='100%'),)
    ])
])

## Callbacks

In [7]:
@app.callback(
    [Output(component_id='modelVersionSelector', component_property='options'),
     Output(component_id='modelVersionSelector', component_property='value'),
     Output(component_id='dateSelector', component_property='date')],
    [Input(component_id='modelSelector', component_property='value')],
    [State(component_id='dateSelector', component_property='date')] 
)
def cargar_modelo(model_name,current_date):
    global modelo
    global configModelo
    global nombre_modelo
    global encoder
    global scalers
    
    if model_name:
        nombre_modelo = model_name
        with open(checkpoints_path + model_name + "/experiment_config.json", 'r') as input_file:
            configModelo = json.load(input_file)
        
        encoder = joblib.load(checkpoints_path + nombre_modelo + '/labelEncoder.pkl')
        scalers = joblib.load(checkpoints_path + nombre_modelo + '/MinMaxScalers.pkl')

        scalers = {k:v for k, v in zip(numeric_feat, scalers)}

        modelo = load_model(checkpoints_path + model_name + "/best.h5")
        model_versions = {x.stem:x.stem.replace('_', ' ') for x in Path(checkpoints_path + model_name).glob('*.h5')}
    else: 
        model_versions={'best':'Best'}

    return (model_versions,'best',current_date)

@app.callback(
    [Output(component_id='placeholder', component_property='children')],
    [Input(component_id='modelVersionSelector', component_property='value')],
    []
)
def load_model_version(model_version):
    global modelo
    global nombre_modelo
    
    modelo = load_model(checkpoints_path + nombre_modelo + "/best.h5")
    return ([],)

@app.callback(
    [Output(component_id='trajectorySelector', component_property='options'),
     Output(component_id='trajectorySelector', component_property='value')],
    [Input(component_id='dateSelector', component_property='date'),
     Input(component_id='airportSelector', component_property='value')],
    [State(component_id='trajectorySelector', component_property='value')]
)
def filter_trajectories(fecha, origen,current_trajectory):
    global month_data
    global current_month
    if fecha:
        m = fecha[:8].replace('-','')

        #if month_data is None or current_month is None or m != current_month:
        month_data = data_loading.load_final_data(m, 'test', sampling=sampling)
        
        # Borrar
        # month_data['sector'] = month_data.track.apply(lambda x: 1 if x >= 180 else 0)
        # month_data['track'] = month_data.track.apply(lambda x: math.sin(x))

        current_month = m
        df = month_data[pd.to_datetime(month_data.timestamp, unit='s').dt.date.astype(str) == fecha].copy()
        if origen:
            df = df[df.aerodromeOfDeparture.isin(origen)]

        labels = df[['fpId','aerodromeOfDeparture']].drop_duplicates().sort_values(['aerodromeOfDeparture'])
        labels = [{'label': f'{x[1].aerodromeOfDeparture} {x[1].fpId}', 'value': x[1].fpId} 
                   for x in labels.iterrows()]
        
        return [labels,current_trajectory]
    else:
        return [{},None]

In [8]:
@app.callback(
    [Output(component_id='windowSelector', component_property='max')],
    [Input(component_id='trajectorySelector', component_property='value')],
    []
)
def prepare_experiment(trajectory):
    global month_data
    global modelo
    global tray_data
    global prediction
    global windows
    global true_windows
    global configModelo
    
    if modelo is None or month_data is None or trajectory is None:
        return [0]
    
    # Full predictions
    tray_data = month_data[month_data.fpId == trajectory].copy()
    prediction = predict_test(tray_data.copy())
    #prediction = modelo.predict(test.copy(), verbose = 0)
    #me quedo con las predicciones que debo pintar
    auxDF = pd.DataFrame()
    
    #### REVISAR
    """
    for i in range(0, len(prediction)-lookforward,lookforward*lookforward):
        auxDF = pd.concat([auxDF, prediction.iloc[list(range(i,i+lookforward))]])
    

    for i in range(0, len(prediction)-lookforward, lookforward):
        # Aplanar la porción de prediction y luego convertirla a un DataFrame
        sub_array = prediction[list(range(i, i+lookforward))]
        sub_array_flattened = sub_array.reshape(-1, sub_array.shape[-1])
        sub_df = pd.DataFrame(sub_array_flattened, columns=feat_dict["objective"])
        
        auxDF = pd.concat([auxDF, sub_df])
    
    #############

    prediction = auxDF.copy()
    prediction['point'] = 'predicted'
    """
    
    lookback = configModelo["lookback"]
    
    # Predictions for individual windows
    windows = [tray_data.iloc[i:i+lookback,:].copy()
               for i in range(tray_data.shape[0]-lookback-lookforward-shift)]
    
    true_windows = [tray_data.iloc[i-1:i+lookforward-1,:].copy()
                    for i in range(lookback+shift+1, tray_data.shape[0])]

    if tray_data is not None:
        max_value = len(windows)-1

    return [max_value]


@app.callback(
    [Output(component_id='sliderDiv', component_property='style')],
    [Input(component_id='experimentTypeSelector', component_property='value')],
    [])
def change_experiment_type(exp_type):
    if exp_type == 'full':
        return ({'display':'none',},)
    else:
        return ({'display':'block',},)



In [9]:
def predict_test(test):
    global modelo
    global configModelo
    global encoder
    global scalers

    categoric_feat = feat_dict.get('categoric')
    numeric_feat = feat_dict.get('numeric')
    objective = feat_dict.get('objective')

    try:
        for columna in categoric_feat:
            test[columna] = encoder.transform(test[columna])#/len(encoder.classes_)
        # Iterar sobre las columnas y escaladores
        for columna in numeric_feat:
            test[columna] = scalers[columna].transform(test[columna])
    except ValueError:
        pass # Si está ya reescalado da un error
        
    test_dataset = crear_dataset(test, configModelo["lookback"], configModelo["batch_size"])

    prediction = modelo.predict(test_dataset, verbose = 0)
    # print(prediction)
    
    # print(prediction.shape)
    for pred in prediction:
        for idx, i in enumerate(objective):
            aux = scalers[i].inverse_transform(pred[:, [idx]])
            pred[:, [idx]] = aux
    # print(prediction.shape)

    # print(prediction.shape)
    # prediction = prediction.reshape(-1, 3) # Esto no es correcto, aplana toda la estructura y mezcla predicciones
    # print(prediction.shape)
    
    # for i in range(len(objective)):
    #     aux = scalers[i].inverse_transform(prediction[:, [i]])
    #     prediction[:, [i]] = aux#.flatten()

    return prediction

def predict_window(window):
    global modelo
    global encoder
    global scalers
    global configModelo

    categoric_feat = feat_dict.get('categoric')
    numeric_feat = feat_dict.get('numeric')
    objective = feat_dict.get('objective')

    window = window[categoric_feat+numeric_feat].copy()
    # print('Ventana original ///////////////////////////')
    # print(window)
    # try:
    for columna in categoric_feat:
        window[columna] = encoder.transform(window[[columna]].values)#/len(encoder.classes_)
    # Iterar sobre las columnas y escaladores
    for columna in numeric_feat:
        window[columna] = scalers[columna].transform(window[[columna]].values)
    
    # except ValueError as E:
    #     pass # Si está ya reescalado da un error
    
    print('Ventana transformada ///////////////////////////')
    print(window)
    prediction = modelo.predict(window.values.reshape(-1,configModelo['lookback'], len(categoric_feat)+len(numeric_feat)), verbose = 0)
    prediction = prediction.reshape(configModelo['lookforward'], len(objective))
    
    print('Predicciones ///////////////////////////')
    print(prediction)
    for idx, i in enumerate(objective):
        prediction[:, [idx]] = scalers[i].inverse_transform(prediction[:,[idx]])
    print('Predicciones desescalado ///////////////////////////')
    print(prediction)
    
    return prediction

@app.callback(
    [Output(component_id='mapGraph', component_property='figure')],
    [Input(component_id='runButton', component_property='n_clicks'),
     Input(component_id='windowSelector', component_property='value'),
     Input(component_id='mapTypeSelector', component_property='value'),
     Input(component_id='mapGraph', component_property='relayoutData')
    ],
    [State(component_id='experimentTypeSelector', component_property='value'),
     State(component_id='mapGraph', component_property='figure')]
)
def run_experiment(clicks, selected_window, map_type, relData, exp_type, current_fig):
    global tray_data
    global prediction
    global windows
    global true_windows
    global modelo
    global feat_dict
    global configModelo
    
    if selected_window is None:
        return (current_fig,)
    
    elif clicks and modelo:
        current_zoom = current_fig['layout']['mapbox']['zoom']
        current_center = current_fig['layout']['mapbox']['center']
        df_viz = tray_data[feat_dict["objective"]].copy()
        df_viz['point'] = 'real'
#3333 ACTUALIZAR       
        if exp_type == 'window':
            #predictions = predict_trajectory_acumulado(windows[selected_window].copy(),lookforward)
            predictions = predict_window(windows[selected_window].copy())

            window = windows[selected_window][objective].copy()
            window['point'] = 'window'
            predictions = pd.DataFrame(predictions.reshape(-1,len(objective)), columns=objective)
            print(predictions)
            predictions['point'] = 'predicted'
            true = true_windows[selected_window][objective].copy()
            true['point'] = 'true'
            
            true_array = true[['latitude','longitude','altitude']].to_numpy()
            true_array = true_array.reshape(-1,len(objective))

            true_array = true_array.reshape((-1, lookforward, len(objective)))
            predictions_array = predictions[objective].to_numpy()
            predictions_array = predictions_array.reshape(-1,len(objective))
            predictions_array = predictions_array.reshape((-1, lookforward, len(objective)))
            
            df_viz = pd.concat([df_viz.iloc[:selected_window], 
                                window,
                                true,
                                predictions,
                                df_viz.iloc[selected_window+configModelo['lookback']+lookforward+shift:]], axis=0)
        elif exp_type == 'full':
            predictions = pd.DataFrame(prediction.reshape(-1,len(objective)), columns=objective)
            predictions['point'] = 'predicted'
            df_viz = pd.concat([df_viz, predictions], axis=0)
            # df_viz = pd.concat([df_viz, predictions], axis=0)
        # print(df_viz)
        # print(df_viz.shape)
        
        fig = px.scatter_mapbox(
            df_viz, 'latitude', 'longitude', height=850, zoom = current_zoom, center = current_center, #zoom=4,
            mapbox_style=map_type, #opacity = 1,
            title='Map',
            color ='point', hover_data = ['altitude']
        )
        # if predictions is not None and exp_type == 'window':
        #     fig.add_trace(go.Scattermapbox(
        #       mode = "markers+lines", lon = predictions.longitude, lat = predictions.latitude,
        #        showlegend=False, marker = {'size': 3, 'color': '#ab63fa'},
        #     ))
        # elif predictions is not None and exp_type == 'full':
        #     fig.add_trace(go.Scattermapbox(
        #       mode = "markers+lines", lon = predictions.longitude, lat = predictions.latitude,
        #        showlegend=False, marker = {'size': 3, 'color': '#ef553b'},
        #     ))
        # px.scatter_geo()
        # points_size = current_zoom*1.5
        # fig.update_traces(marker={'size':points_size})
        return (fig,)
    else:
        return (px.scatter_mapbox(lat=[43.0],lon=[4.0], zoom = 4, mapbox_style='open-street-map', title='Map'),)

## Ejecución

In [10]:
import warnings
warnings.filterwarnings("ignore")

In [11]:
app.run(port= 8051)
# mode='inline'


Ventana transformada ///////////////////////////
     aerodromeOfDeparture  aerodromeOfDestination  latitude  longitude  \
801                  4033                    8885  0.765882   0.417600   
802                  4033                    8885  0.765775   0.417484   
803                  4033                    8885  0.765449   0.417151   
804                  4033                    8885  0.765080   0.416770   
805                  4033                    8885  0.764786   0.416471   
806                  4033                    8885  0.764391   0.416053   
807                  4033                    8885  0.763985   0.415625   
808                  4033                    8885  0.763541   0.415137   
809                  4033                    8885  0.763284   0.414488   
810                  4033                    8885  0.763445   0.413819   
811                  4033                    8885  0.764129   0.413183   
812                  4033                    8885  0.764925   